# Simple Reinforcement Learning with Tensorflow Part 8: Asynchronus Advantage Actor-Critic (A3C)

This iPython notebook includes an implementation of the [A3C algorithm](https://arxiv.org/pdf/1602.01783.pdf). In it we use A3C to solve a simple 3D Doom challenge using the [VizDoom engine](http://vizdoom.cs.put.edu.pl/). For more information on A3C, see the accompanying [Medium post](https://medium.com/p/c88f72a5e9f2/edit).

This tutorial requires that VizDoom is installed. It can be easily obtained with:

`pip install vizdoom`

We also require `basic.wad` and `helper.py`, both of which are available from the [DeepRL-Agents github repo](https://github.com/awjuliani/DeepRL-Agents).

While training is taking place, statistics on agent performance are available from Tensorboard. To launch it use:

`tensorboard --logdir=worker_0:'./train_0',worker_1:'./train_1',worker_2:'./train_2',worker_3:'./train_3'`

In [1]:
import pickle
with open("p_experiments.snake","rb") as f:
    exps = pickle.load(f)

In [2]:
from __future__ import division

import SnakeEnvironment
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import sys, os
print(sys.version_info)
import multiprocessing
import threading
import shutil

sys.version_info(major=3, minor=4, micro=6, releaselevel='final', serial=0)



### Parameters

In [3]:
OUTPUT_GRAPH = True
LOG_DIR = './log'
N_WORKERS = 6 #multiprocessing.cpu_count()
MAX_GLOBAL_EP = 50000
GLOBAL_NET_SCOPE = 'Global_Net'
UPDATE_GLOBAL_ITER = 89
GAMMA = 0.995
ENTROPY_BETA = 0.01
LR_A = 1e-4    # learning rate for actor
LR_C = 1e-4    # learning rate for critic
GLOBAL_REWARD = []
GLOBAL_EP = 0
model_path = './model_offlineA3C'
load_model = False
episode_length = 89


s_size = 7
a_size = 9

os.system('rm -Rf log train_W* '+model_path)
os.makedirs(model_path)

## A3C Approach

### Implementing the Actor-Critic network

In [4]:
class ACNet(object):
    def __init__(self, scope, globalAC=None):

        if scope == GLOBAL_NET_SCOPE:   # Only need parameters of global network
            with tf.variable_scope(scope):
                self.s = tf.placeholder(tf.float32, [None, s_size], 'S')
                self._build_net()
                self.a_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/actor')
                self.c_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/critic')
        else:   # Uselocal net to calculate losses
            with tf.variable_scope(scope):
                self.s = tf.placeholder(tf.float32, [None, s_size], 'S')
                self.a_his = tf.placeholder(tf.int32, [None, ], 'A')
                self.v_target = tf.placeholder(tf.float32, [None, 1], 'Vtarget')
                
                #Get policy(a_prob) and value(v) from actor, critic net
                self.a_prob, self.v = self._build_net()

                td = tf.subtract(self.v_target, self.v, name='TD_error')
                with tf.name_scope('q_loss'):
                    self.c_loss = tf.reduce_mean(tf.square(td))

                with tf.name_scope('a_loss'):
                    #use clip to avoid log(0): tf.clip_by_value(self.policy, 1e-20, 1.0)
                    log_prob = tf.reduce_sum(tf.log(tf.clip_by_value(self.a_prob, 1e-20, 1.0)) * tf.one_hot(self.a_his, a_size, dtype=tf.float32), axis=1, keep_dims=True)
                    exp_v = log_prob * td
                    #Found someone use entropy to encourage exploration
                    #larger entropy means more stochastic actions
                    entropy = -tf.reduce_sum(self.a_prob * tf.log(tf.clip_by_value(self.a_prob, 1e-20, 1.0)), axis=1, keep_dims=True)  # encourage exploration
                    self.exp_v = ENTROPY_BETA * entropy + exp_v
                    self.a_loss = tf.reduce_mean(-self.exp_v)

                with tf.name_scope('local_grad'):
                    self.a_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/actor')
                    self.c_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/critic')
                    self.a_grads = tf.gradients(self.a_loss, self.a_params)
                    self.c_grads = tf.gradients(self.c_loss, self.c_params)
                
            # Syncronization
            with tf.name_scope('sync'):
                #assign global parameters to local parameters
                with tf.name_scope('pull'):
                    self.pull_a_params_op = [local_param.assign(global_param) for local_param, global_param in zip(self.a_params, globalAC.a_params)]
                    self.pull_c_params_op = [local_param.assign(global_param) for local_param, global_param in zip(self.c_params, globalAC.c_params)]
                #update params of global net by pushing the calculated gradients of local net to global net    
                with tf.name_scope('push'):
                    self.update_a_op = trainer_A.apply_gradients(zip(self.a_grads, globalAC.a_params))
                    self.update_c_op = trainer_C.apply_gradients(zip(self.c_grads, globalAC.c_params))

    def _build_net(self):
        w_init = tf.random_normal_initializer(0., .1)
        with tf.variable_scope('actor'):
            #dense layer is normal NN,see: https://www.tensorflow.org/api_docs/python/tf/layers/dense
            layer4 = tf.layers.dense(self.s, a_size, tf.nn.relu6, kernel_initializer=w_init, name='layer4')
            layer3 = tf.layers.dense(layer4, a_size, tf.nn.relu6, kernel_initializer=w_init, name='layer3')
            layer2 = tf.layers.dense(layer3, a_size, tf.nn.relu6, kernel_initializer=w_init, name='layer2')
            a_prob = tf.layers.dense(layer2, a_size, tf.nn.softmax, kernel_initializer=w_init, name='ap')
        with tf.variable_scope('critic'):
            v4 = tf.layers.dense(self.s, 1, tf.nn.relu6, kernel_initializer=w_init, name='v4')#relu6 is better than relu
            v3 = tf.layers.dense(v4, 1, tf.nn.relu6, kernel_initializer=w_init, name='v3')#relu6 is better than relu
            v2 = tf.layers.dense(v3, 1, tf.nn.relu6, kernel_initializer=w_init, name='v2')#relu6 is better than relu
            v = tf.layers.dense(v2, 1, tf.nn.relu6, kernel_initializer=w_init, name='v')#relu6 is better than relu
        return a_prob, v

    def update_global(self, feed_dict):  # run by a local
        a_grads,c_grads,c_loss,a_loss,_,_ = SESS.run([self.a_grads,self.c_grads,self.c_loss, self.a_loss,self.update_a_op, self.update_c_op], feed_dict)# local grads applies to global net
        return a_loss,c_loss,a_grads,c_grads
    def pull_global(self):  # run by a local
        SESS.run([self.pull_a_params_op, self.pull_c_params_op])

    def choose_action(self, s):  # run by a local
        prob_weights = SESS.run(self.a_prob, feed_dict={self.s: np.array(s)})
        action = np.random.choice(range(prob_weights.shape[1]),
                                  p=prob_weights.ravel())  # select action w.r.t the actions prob
        return action

## Worker Agent

In [5]:
class Worker(object):
    def __init__(self, name, workerID, globalAC):
        self.workerID = workerID
        self.env = SnakeEnvironment.SnakeEnv(exps,workerID,89)
        self.name = name
        self.AC = ACNet(name, globalAC)
        self.model_path = model_path
        self.summary_writer = tf.summary.FileWriter("train_"+str(self.name))        

    def work(self):
        global GLOBAL_REWARD, GLOBAL_EP
        total_step = 1
        buffer_s, buffer_a, buffer_r = [], [], []
        while not COORD.should_stop() and GLOBAL_EP < MAX_GLOBAL_EP:
            s,_,_ = self.env.reset()
            ep_r = 0
            ep_step=0
            ep_action_counter=0
            a_loss, c_loss = [], []            
            while True:
                a,_ = self.env.action_space.sample()
                s1, r, done,_ = self.env.step(a)
                ep_r += r
                buffer_s.append(s)
                buffer_a.append(a)
                buffer_r.append(r)

                if total_step % UPDATE_GLOBAL_ITER == 0:   # update global and assign to local net
                    if done:
                        v_s1 = 0   # terminal
                    else:
                        v_s1 = SESS.run(self.AC.v, {self.AC.s: np.array(s1)})[0, 0]
                        assert not(np.any(np.isnan(v_s1)))
                    buffer_v_target = []
                    for r in buffer_r[::-1]:    # reverse buffer r
                        v_s1 = r + GAMMA * v_s1
                        buffer_v_target.append(v_s1)
                    buffer_v_target.reverse()

                    buffer_s, buffer_a, buffer_v_target = np.vstack(buffer_s), np.array(buffer_a), np.vstack(buffer_v_target)
                    feed_dict = {
                        self.AC.s: buffer_s,
                        self.AC.a_his: buffer_a,
                        self.AC.v_target: buffer_v_target,
                    }
                    a_l,c_l,a_grads,c_grads = self.AC.update_global(feed_dict)
                    a_loss.append(a_l)
                    c_loss.append(c_l)

                    buffer_s, buffer_a, buffer_r = [], [], []
                    self.AC.pull_global()

                s = s1
                total_step += 1
                ep_step +=1
                if ep_step == episode_length:
                    GLOBAL_REWARD.append(ep_r)

                    print(
                        self.name,
                        "Ep:", GLOBAL_EP,
                        "| Ep_r: %i" % GLOBAL_REWARD[-1]
                          )
                    if GLOBAL_EP % 100 == 0:
                        saver.save(SESS,self.model_path+'/model-'+str(GLOBAL_EP)+'.cptk')
                        print ("Saved Model")
                    if GLOBAL_EP % 5 ==0:
                        mean_reward = np.mean(GLOBAL_REWARD[-5:])
                        mean_c_loss = np.max(np.mean(c_loss[-5:]))
                        mean_a_loss = np.max(np.mean(a_loss[-5:]))
                        #mean_a_grads = np.mean(a_grads[-5:])
                        #mean_value = np.mean(self.episode_mean_values[-5:])
                        summary=tf.Summary()
                        summary.value.add(tag='Perf/Reward', simple_value=float(mean_reward))
                        #summary.value.add(tag='Perf/Value', simple_value=float(mean_value))
                        summary.value.add(tag='Losses/Value Loss', simple_value=float(mean_c_loss))
                        summary.value.add(tag='Losses/Policy Loss', simple_value=float(mean_a_loss))
                        #summary.value.add(tag='Losses/Entropy', simple_value=float(e_l))
                        #summary.value.add(tag='Losses/Grad Norm', simple_value=float(mean_a_grads))
                        #summary.value.add(tag='Losses/Var Norm', simple_value=float(c_grads))
                        self.summary_writer.add_summary(summary, GLOBAL_EP)

                        self.summary_writer.flush()
                    GLOBAL_EP += 1
                    
                    break

### Training the network

In [6]:
if __name__ == "__main__":
    os.system("rm -r train_W*")
    SESS = tf.Session()

    with tf.device("/cpu:0"):
        trainer_A = tf.train.AdamOptimizer(LR_A, name='RMSPropA')
        trainer_C = tf.train.AdamOptimizer(LR_C, name='RMSPropC')
        GLOBAL_AC = ACNet(GLOBAL_NET_SCOPE)  # we only need its params
        workers = []
        # Create worker
        for i in range(N_WORKERS):#N_WORKERS
            i_name = 'W_%i' % i   # worker name
            workers.append(Worker(i_name, i, GLOBAL_AC))
        saver = tf.train.Saver(max_to_keep=5)

    COORD = tf.train.Coordinator()
    if load_model==True:
        print ('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(SESS,ckpt.model_checkpoint_path)
    else:
        SESS.run(tf.global_variables_initializer())

    if OUTPUT_GRAPH:
        if os.path.exists(LOG_DIR):
            shutil.rmtree(LOG_DIR)
        tf.summary.FileWriter(LOG_DIR, SESS.graph)

    worker_threads = []
    for worker in workers:
        job = lambda: worker.work()
        t = threading.Thread(target=job)
        t.start()
        worker_threads.append(t)
    COORD.join(worker_threads)

    plt.plot(np.arange(len(GLOBAL_REWARD)), GLOBAL_REWARD)
    plt.xlabel('step')
    plt.ylabel('Total moving reward')
    plt.show()

W_0 Ep: 0 | Ep_r: 0
W_3 Ep: 0 | Ep_r: 0
W_1 Ep: 0 | Ep_r: 1
W_2 Ep: 0 | Ep_r: 1
W_4 Ep: 0 | Ep_r: 0
W_5 Ep: 0 | Ep_r: 1
Saved Model
Saved Model
W_2 Ep: 2 W_1| Ep_r: 0 
Ep: 2 | Ep_r: 1
W_1 Ep: 4 | Ep_r: 2
W_2W_1  Ep:Ep:  55  | Ep_r: 2| Ep_r: 3

W_2 Ep: 7 | Ep_r: 1
W_1Saved Model
Saved Model Ep:
 8 | Ep_r: 1
W_2 Ep: 11 | Ep_r: 0
W_1 Ep: 12 | Ep_r: 0
W_2W_0 Ep:  13Ep:  | Ep_r: 313
 | Ep_r: 0
W_3 Ep: 15 | Ep_r: 0
W_1 Ep: 16 | Ep_r: 1
W_0W_2 Ep:  Ep:17 17 | Ep_r: 5
 | Ep_r: 2
W_3 W_0Ep: W_1 Ep: 19 | Ep_r: 0
 19 Ep:| Ep_r: 1 20 
| Ep_r: 1
W_3 W_2 Ep: Ep:20  21| Ep_r: 0 
| Ep_r: 2
W_2 Ep:W_0  24Ep:Saved Model
 24 | Ep_r: 0
 | Ep_r: 0
W_1 Ep: 27 | Ep_r: 0
W_0 Ep: 28 | Ep_r: 0
W_2W_3 Ep:  Ep:29  29Saved Model| Ep_r: 0 

| Ep_r: 2
W_4 Ep: 32 | Ep_r: 0
W_0 Ep: 33 | Ep_r: 0
W_2W_1 Ep:  Ep:W_534   34Ep:| Ep_r: 0  
| Ep_r: 535W_4 
 Ep:| Ep_r: 1 
37 | Ep_r: 0
W_3 Ep: 38 | Ep_r: 3
W_0 Ep: 39 | Ep_r: 0
W_5 Ep: 40 | Ep_r: 1
W_1 Ep:W_2  41Ep:  | Ep_r: 041
 | Ep_r: 2
W_4W_3 Ep:  43Ep:  | Ep_r: 243
 | Ep_r

W_0 W_1Ep:  379Ep:W_3   | Ep_r: 1380Ep:
  381| Ep_r: 0 
| Ep_r: 0
W_5 Ep: 382 | Ep_r: 0
W_4 Ep: 383W_2  | Ep_r: 0Ep:
 384 | Ep_r: 3
W_1W_0  Ep:Ep:  385385  | Ep_r: 2| Ep_r: 0

W_4 Ep: 387 | Ep_r: 0
W_3 Ep: 388 | Ep_r: 0
W_2 W_5Ep:  Ep:389  389| Ep_r: 1 
| Ep_r: 1
W_4 Ep: 391 W_1| Ep_r: 1 
Ep: 392 | Ep_r: 0W_0
 W_3 Ep:Ep: W_2 393 393 Ep: | Ep_r: 0 W_5| Ep_r: 1
395 
Ep: | Ep_r: 0 
396 | Ep_r: 2
W_1 Ep: 397 | Ep_r: 1
W_0 W_4Ep:  Ep:398  398| Ep_r: 0 
| Ep_r: 1
W_3W_2 W_5 Ep: Ep: Ep: 400 400 400 | Ep_r: 3 | Ep_r: 2
| Ep_r: 1

W_4 Ep:W_0W_1 Ep:  Ep:400   | Ep_r: 0400400
  | Ep_r: 0
| Ep_r: 0
Saved ModelSaved Model

Saved Model
W_3W_5 Ep:  Ep:403  403| Ep_r: 2
 | Ep_r: 1
W_2 Ep: 404 | Ep_r: 0
W_3 Ep: 406 | Ep_r: 1
W_5W_2 Ep:  Ep:407  | Ep_r: 1
407 | Ep_r: 0
W_3 Ep: 409 | Ep_r: 4
W_2 Ep: 410 | Ep_r: 2
W_3 W_5 Ep:Ep:  411411  | Ep_r: 0| Ep_r: 1

W_2 Ep: 413 | Ep_r: 2
W_5 Ep: 414 | Ep_r: 0
W_3 Ep: 415 | Ep_r: 0
W_2 Ep: 416 | Ep_r: 1
W_5 W_2Ep:  Ep:417  417| Ep_r: 1 
| Ep_r: 5
W_2 Ep: 419 | Ep_r

W_4 Ep: 762 | Ep_r: 2
W_2 W_3Ep: W_5 Ep: 763 Ep: 764 | Ep_r: 1 763
| Ep_r: 0 | Ep_r: 1

W_0 Ep: W_1766  Ep:| Ep_r: 1 
766 | Ep_r: 1
W_4 W_5W_3Ep:   Ep:Ep:768   769770| Ep_r: 2  
| Ep_r: 1| Ep_r: 3

W_2 Ep: 771 | Ep_r: 0
W_0 Ep: W_4772 | Ep_r: 0
 Ep: 773 | Ep_r: 3
W_1W_5  Ep:Ep:W_3   774774Ep:   | Ep_r: 3| Ep_r: 0776

 | Ep_r: 2
W_2 Ep: 777 | Ep_r: 0W_0
 Ep: 778 | Ep_r: 3
W_5 W_1Ep: W_3W_4 Ep: 779 Ep: 779 | Ep_r: 2 780
| Ep_r: 1 | Ep_r: 0

 Ep: 780 | Ep_r: 0
W_2W_1W_0  Ep:Ep: W_4  Ep: 783783   783| Ep_r: 1| Ep_r: 0 

| Ep_r: 1
Ep: W_5785  W_3| Ep_r: 0Ep: 
 Ep: 786787  | Ep_r: 0| Ep_r: 1

W_0 Ep: W_1789 W_2W_3W_4 Ep:  | Ep_r: 0 Ep:Ep: 
790  Ep:W_5  792791 Ep:| Ep_r: 1  791 
| Ep_r: 0| Ep_r: 0 794| Ep_r: 0

 
| Ep_r: 0
W_0 Ep: 795 | Ep_r: 0
W_2W_3 Ep:  796Ep:W_1   | Ep_r: 2796Ep:
  W_4 | Ep_r: 2798W_5Ep:
   Ep:| Ep_r: 0799 
 800| Ep_r: 0 
| Ep_r: 2
W_0 W_3 Ep:Ep:  800800  | Ep_r: 0| Ep_r: 2

W_2 Ep: 800 | Ep_r: 1
Saved Model
W_4W_1  Ep:Ep:  801801  | Ep_r: 2
| Ep_r: 1
W_5 Ep: 803 | Ep_r: 

W_3 Ep: 1116 | Ep_r: 3
W_3W_0  Ep:Ep:  11171117  | Ep_r: 2| Ep_r: 0

W_0 W_3Ep:  Ep:1119  1120| Ep_r: 1 
| Ep_r: 1
Saved Model
Saved Model
W_3 Ep: 1123W_0  | Ep_r: 0Ep:
 1123 | Ep_r: 0
Saved Model
W_0 Ep: 1126W_1 Ep:  | Ep_r: 21126W_2
 | Ep_r: 4
 Ep: 1126 | Ep_r: 0
W_3 Ep: 1129W_4  | Ep_r: 2Ep:
 1130 | Ep_r: 1
W_0W_1 Ep: W_2  1131Ep:Ep:   | Ep_r: 111311131
  | Ep_r: 7| Ep_r: 0

Saved Model
W_4 Ep: 1135 | Ep_r: 0
W_3 Ep: 1136 | Ep_r: 1
W_1 Ep: W_01137 W_4 Ep: | Ep_r: 2 Ep:W_5
1138  W_2  1139Ep:Ep:| Ep_r: 1   
| Ep_r: 111401140 
 | Ep_r: 1| Ep_r: 0

W_3 Ep: 1142 | Ep_r: 1
W_1 Ep: 1143 W_5| Ep_r: 1 
Ep: 1144 | Ep_r: 0
W_0W_2 W_4 Ep: Ep: Ep: 1145 1145 1145 | Ep_r: 3 | Ep_r: 1
| Ep_r: 0

W_3 Ep: W_51148  Ep:| Ep_r: 2 
1149 | Ep_r: 4
W_1 Ep: 1150 | Ep_r: 0
W_4 Ep:W_2  W_01151Ep:  | Ep_r: 21152
 | Ep_r: 0
 Ep: 1153 | Ep_r: 2
W_3W_5  Ep:Ep:W_1   11541155Ep:   | Ep_r: 2| Ep_r: 11156

 | Ep_r: 0
W_2W_4  Ep: 1157Ep:  | Ep_r: 01157
 | Ep_r: 0
W_0 Ep: 1159 | Ep_r: 0
W_5 Ep: 1160W_3W_1   | Ep_r: 0Ep

W_3 Ep: 1478 | Ep_r: 0
W_0 Ep: 1479 | Ep_r: 0
W_2 Ep: 1480 | Ep_r: 0
W_4 Ep: 1481 | Ep_r: 1
W_1 Ep: 1482 | Ep_r: 3
W_5W_3 Ep:  Ep:1483 W_0 1483 | Ep_r: 1 Ep:
| Ep_r: 1 1485
 | Ep_r: 1
W_1 Ep: 1486 | Ep_r: 0
W_2 Ep: 1487 | Ep_r: 0
W_4 Ep: 1488 | Ep_r: 0
W_0 Ep: 1489 | Ep_r: 0
W_5 Ep: 1490 | Ep_r: 0W_3
 Ep: 1491 | Ep_r: 0
W_1 W_2Ep:  Ep:1492  1492| Ep_r: 1 
| Ep_r: 6
W_5 W_4W_0Ep:   Ep:Ep:1494   14941495| Ep_r: 1  
| Ep_r: 0| Ep_r: 1

W_3W_2 Ep: 1497 | Ep_r: 1
 Ep: 1497 | Ep_r: 1
W_1 Ep: 1499 | Ep_r: 1
W_4 Ep:W_5  1500Ep:  | Ep_r: 11500
 | Ep_r: 1
W_2 W_0 Ep: 1500 | Ep_r: 0
Ep: 1500 | Ep_r: 1W_3
 Ep: 1500 | Ep_r: 0
W_1 Ep: 1500 | Ep_r: 0
Saved Model
W_5 Ep: 1501 | Ep_r: 0
Saved Model
W_5 Ep: 1503 | Ep_r: 1
W_5 Ep:W_0 Ep:  15041504  | Ep_r: 0| Ep_r: 5

W_0 Ep: 1506 | Ep_r: 0
W_5 Ep: 1507 | Ep_r: 3
W_0 Ep: 1508 | Ep_r: 2
W_5 Ep: 1509W_0  | Ep_r: 0Ep:
 1510 | Ep_r: 0
W_5W_0 Ep: 1511  Ep:| Ep_r: 0 
1511 | Ep_r: 0
Saved Model
W_0 W_5Ep:  Ep:1514 1514 | Ep_r: 0
 | Ep_r: 0
W_4 Ep: 1516 | Ep_r: 

...

Saved Model
W_0 Ep: 1601 | Ep_r: 0
Saved Model


Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/lib64/python3.4/threading.py", line 911, in _bootstrap_inner
    self.run()
  File "/usr/lib64/python3.4/threading.py", line 859, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-a7ccd3289f34>", line 31, in <lambda>
    job = lambda: worker.work()
  File "<ipython-input-5-c1e38f1ef008>", line 35, in work
    assert not(np.any(np.isnan(v_s1)))
AssertionError

Exception in thread Thread-14:
Traceback (most recent call last):
  File "/usr/lib64/python3.4/threading.py", line 911, in _bootstrap_inner
    self.run()
  File "/usr/lib64/python3.4/threading.py", line 859, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-a7ccd3289f34>", line 31, in <lambda>
    job = lambda: worker.work()
  File "<ipython-input-5-c1e38f1ef008>", line 35, in work
    assert not(np.any(np.isnan(v_s1)))
AssertionError



W_3 Ep: 1603 | Ep_r: 0
Saved ModelSaved Model

Saved Model
W_4 Ep: 1607 | Ep_r: 0
W_1 Ep: 1608 | Ep_r: 1
W_5 Ep: 1609 | Ep_r: 0


Exception in thread Thread-15:
Traceback (most recent call last):
  File "/usr/lib64/python3.4/threading.py", line 911, in _bootstrap_inner
    self.run()
  File "/usr/lib64/python3.4/threading.py", line 859, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-a7ccd3289f34>", line 31, in <lambda>
    job = lambda: worker.work()
  File "<ipython-input-5-c1e38f1ef008>", line 35, in work
    assert not(np.any(np.isnan(v_s1)))
AssertionError

Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib64/python3.4/threading.py", line 911, in _bootstrap_inner
    self.run()
  File "/usr/lib64/python3.4/threading.py", line 859, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-a7ccd3289f34>", line 31, in <lambda>
    job = lambda: worker.work()
  File "<ipython-input-5-c1e38f1ef008>", line 35, in work
    assert not(np.any(np.isnan(v_s1)))
AssertionError

Exception in thread Thread-16:
Traceback (most recent call las

KeyboardInterrupt: 

In [ ]:
import pickle

load_model = True #Whether to load a saved model.
#model_path = "./model_Test" #The path to save our model to.

#saver.save(SESS,'./model_online/model-'+str(GLOBAL_EP)+'.cptk')\
with open('runData.snake', 'wb') as file:
    data = {}
    data['amps'] = []
    data['spFreqs'] = []
    data['GLOBAL_EP'] = MAX_GLOBAL_EP
    pickle.dump(data, file)